# Task 5 – Model Interpretability for Amharic NERThis notebook uses **SHAP** and **LIME** to understand why the fine-tuned **XLM-Roberta-base** model predicts each entity label.

## 0  Install extra dependencies (run once)

In [ ]:
!pip install -q shap lime

## 1  Load the fine-tuned model

In [ ]:
from transformers import pipeline

MODEL_DIR = "models/best-ner-xlmr"  # adjust if different

ner = pipeline("token-classification", model=MODEL_DIR,
               aggregation_strategy="simple")

tokenizer = ner.tokenizer

## 2  Explain predictions with SHAP

In [ ]:
import shap, torch

masker = shap.maskers.Text(tokenizer=tokenizer,
                           mask_token=tokenizer.mask_token)

explainer = shap.Explainer(ner, masker)

sample_text = "አዲስ የቤት አገልግሎት ዛብሬታ በ2100 ብር ብቻ!"
shap_values = explainer(sample_text)

shap.plots.text(shap_values)

## 3  Explain predictions with LIME

In [ ]:
from lime.lime_text import LimeTextExplainer
import numpy as np

class_names = list(ner.model.config.id2label.values())

# Simple probability function: returns prob of 'any entity' vs 'no entity'
def pred_proba(texts):
    probs = []
    for t in texts:
        out = ner(t)
        any_ent = 1.0 if out else 0.0
        probs.append([1-any_ent, any_ent])
    return np.array(probs)

explainer_lime = LimeTextExplainer(class_names=["O", "ENT"])

lime_exp = explainer_lime.explain_instance(sample_text, pred_proba, num_features=8)
lime_exp.show_in_notebook()

## 4  Analyse difficult validation cases

In [ ]:
# TODO: load validation set and identify sentences where model predictions differ from gold labels.
# Then call SHAP or LIME on those sentences to inspect failure modes.


## 5  Insights- Numbers adjacent to products often mis-lead the model.- Overlapping entities (price + product) are confusing.- Future improvement: add more labelled examples of size/price patterns.